In [5]:
import pandas as pd
import numpy as np

import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Switch area
SUBJECT = 'material_garment'
VOCAB_SIZE = 900

AttributeError: module 'nltk' has no attribute 'data'

In [ ]:
BOM = 'DataSource/Original_BOM/AB18MJ1_032_ BOM.xlsx'
VOCAB = 'DataSource/Trained Data/' + SUBJECT + '_vocabulary.csv'
TRAIN_DATA_0 = 'DataSource/Trained Data/' + SUBJECT + '_prob_tokens_ctg_0_in_train_data'
TRAIN_DATA_1 = 'DataSource/Trained Data/' + SUBJECT + '_prob_tokens_ctg_1_in_train_data'
TRAIN_DATA_ALL =  'DataSource/Trained Data/' + SUBJECT + '_prob_tokens_all_in_train_data'
PROB_1_TRAIN_DATA = 'DataSource/Trained Data/' + SUBJECT + '_prob_ctg_1_in_train_data'

In [ ]:
TRAIN_DATA_0

In [ ]:
vocab = pd.read_csv(VOCAB, index_col = 0)

In [ ]:
bom = pd.read_excel(BOM, index_col=None, header=None)

* Delete the none columns 

In [ ]:
def delete_col(bom):
    for col in bom:
        if bom[col].count() == 0:
            bom = bom.drop(col, axis = 1)
        
    return bom

In [ ]:
bom = delete_col(bom)

In [ ]:
bom

## Nltk stemmered Function

In [ ]:
def stemmered_nltk_convert(col_of_df):
    '''
    Parameter of this function is a column of a dataFrame.
    
    '''
    # difine Stop words
    stop_words = set(stopwords.words('english'))
    # Difine Stemmer
    stemmer = SnowballStemmer('english')
    
    # converts to lower case and splits up the words
    words = word_tokenize(col_of_df)
    filtered_words = []
    
    for word in words:
        # Removes the stop words and punctuation
        # if word is not in the stop_words list and is not a alpha.
        if word not in stop_words and word.isalpha():
            filtered_words.append(stemmer.stem(word))
            
    return filtered_words

## Walk through bom
* Parse a xlsm of bom, turn the row into a col.
* Put the col to the classify function
* Return the index of row that is True

In [ ]:
def turn_series(bom):
    '''
    
    The parameter of the func is a dataFrame
    
    '''
    database = []
    
    for row in bom.index:
        row_str = str()
        for col in bom:
            row_str = row_str + ', ' + str(bom.at[row, col])
        database.append(row_str)
    
    col = pd.Series(database)
#     index_list = classify_series(col)
    
    return col

* Turn the bom into series contains of string list

In [ ]:
new_bom = turn_series(bom)

In [ ]:
new_bom

In [ ]:
type(new_bom)

* Convert the series of bom into Stemmed bom

In [ ]:
%%time
stemmed_bom = new_bom.apply(stemmered_nltk_convert)

In [ ]:
stemmed_bom[54]

* Convert the Stemmed bom into df

In [ ]:
word_col_df = pd.DataFrame.from_records(stemmed_bom.tolist())
word_col_df

### Sparse Matrix Function
* Create a sparse Matrix for the data we want to predict
* The difference of this function in comparition with Classification Model for Train data, is this function don't need CATEGORY.

In [ ]:
def make_sparse_matrix(df, vocabulary):
    """
    Param1:
    The data we want to sparse, which must be in format of DataFrame.
    Param2:
    The vocabulary, it is generated when we training datas.
    
    Returns a sparse matrix as dataframe
    """
 
    indexed_words = pd.Index(vocabulary.VOCAB_WORD)
    nr_rows = df.shape[0]
    nr_cols = df.shape[1]
    word_set = set(indexed_words)
    dict_list = []
    
    for i in range(nr_rows):
        for j in range(nr_cols):
            
            word = df.iat[i, j]
            if word in word_set:
                doc_id = df.index[i]
                word_id = indexed_words.get_loc(word)
                
                item = {'MATERIAL_ID': doc_id,
                       'OCCURENCE': 1, 'WORD_ID': word_id}
                
                dict_list.append(item)
                
    return pd.DataFrame(dict_list)

* Sparse the data we want to predict

In [ ]:
%%time
sparse_predict_df = make_sparse_matrix(word_col_df, vocab)

In [ ]:
sparse_predict_df

* Grouped by MATERIAL_ID

In [ ]:
sparse_predict_df_grouped = sparse_predict_df.groupby(['MATERIAL_ID', 'WORD_ID']).sum()
sparse_predict_df_grouped

In [ ]:
sparse_predict_df_grouped = sparse_predict_df_grouped.reset_index()
sparse_predict_df_grouped

In [ ]:
np.savetxt('DataSource/Trained Data/' + SUBJECT + '_sparse_predict_data', sparse_predict_df_grouped, fmt = '%d')

In [ ]:
PREDICT_DATA = 'DataSource/Trained Data/' + SUBJECT + '_sparse_predict_data'

In [ ]:
sparse_predict_data = np.loadtxt(PREDICT_DATA, delimiter = ' ', dtype = int)

## Full Matrix
* Since we want to predict the data, so we create the Full Feature directly.

In [ ]:
def make_full_feature(sparse_matrix, nr_words, doc_idx = 0, word_idx = 1, freq_idx = 2):
    column_names = ['MATERIAL_ID'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:,0])
    full_matrix = pd.DataFrame(index = doc_id_names, columns = column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'MATERIAL_ID'] = doc_nr
        full_matrix.at[doc_nr, word_id] = occurrence
        
    full_matrix.set_index('MATERIAL_ID', inplace = True)
    return full_matrix

In [6]:
%%time
predict_full_feature = make_full_feature(sparse_predict_data, VOCAB_SIZE)

NameError: name 'make_full_feature' is not defined

In [7]:
predict_full_feature

NameError: name 'predict_full_feature' is not defined

# Joint probability

* Import training data, token probability of category 1, 0, and in both category 

In [ ]:
train_data_1 = np.loadtxt(TRAIN_DATA_1)
train_data_0 = np.loadtxt(TRAIN_DATA_0)
train_data_all = np.loadtxt(TRAIN_DATA_ALL)

* Joint probability
* Take the currence of each token in the data we want to analyze to multiply the trained probability of token

In [ ]:
joint_prob_1 = predict_full_feature.dot(train_data_1)# Joint probability in log format
joint_prob_0 = predict_full_feature.dot(train_data_0)

# Joint probability in log format

* Import training data, token probability of category 1, 0, and in both category 

In [ ]:
train_data_1 = np.loadtxt(TRAIN_DATA_1)
train_data_0 = np.loadtxt(TRAIN_DATA_0)
train_data_all = np.loadtxt(TRAIN_DATA_ALL)

* Get the probability of category 1 in train_set

In [ ]:
prob_ctg_1 = pd.read_csv(PROB_1_TRAIN_DATA, index_col = 0)

In [ ]:
prob_ctg_1_train_data = prob_ctg_1.loc[0, 'prob_ctg_1_train_set']

In [ ]:
prob_ctg_1_train_data

* Log format

In [ ]:
joint_log_ctg_1 = predict_full_feature.dot(np.log(train_data_1) - np.log(train_data_all)) + np.log(prob_ctg_1_train_data)

In [ ]:
joint_log_ctg_0 = predict_full_feature.dot(np.log(train_data_0)-np.log(train_data_all))+np.log(1 - prob_ctg_1_train_data)

In [ ]:
prediction_log = joint_log_ctg_1 > joint_log_ctg_0

In [ ]:
prediction_log

# Prediction

* Normal format of Joint probability 

In [ ]:
prediction = joint_prob_1 > joint_prob_0

In [ ]:
row_list = prediction[prediction == True].index

In [ ]:
row_list

In [ ]:
bom.loc[row_list,:]

* Log format joint probability 

In [ ]:
prediction = joint_log_ctg_1 > joint_log_ctg_0

In [ ]:
row_list = prediction[prediction == True].index

In [ ]:
row_list

In [ ]:
bom.loc[row_list,:]

In [ ]:
bom.to_csv('result/bom_for_check_column_' + BOM[24:-5] + '.csv')

# Conclution
* The log format seems more precise in prediction.